<a href="https://colab.research.google.com/github/dsfdev2023/studentEvaluatiuon/blob/main/Streamlit_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install necessary packages:**

In [16]:
!pip install PyPDF2 tabulate streamlit


In [17]:
!pip install pyngrok


In [22]:
!pip install pandas openpyxl


# **Modify the code for Streamlit**

In [23]:
%%writefile streamlit_app.py

import streamlit as st
import pandas as pd
import PyPDF2
import re
import os
from io import BytesIO
import base64

# Create a function to generate download links
def to_excel(df):
    output = BytesIO()
    writer = pd.ExcelWriter(output, engine='openpyxl')
    df.to_excel(writer, index=False, sheet_name='Sheet1')
    writer.save()
    processed_data = output.getvalue()
    return processed_data

def get_table_download_link(df, filename='data.xlsx', text='Download Excel file'):
    val = to_excel(df)
    b64 = base64.b64encode(val)  # val looks like b'...'
    return f'<a href="data:application/octet-stream;base64,{b64.decode()}" download="{filename}">{text}</a>'

# App Title
st.title("PDF Data Extraction Tool")

uploaded_files = st.file_uploader("Upload PDF files", type=["pdf"], accept_multiple_files=True)

results = []
headers = ["File Name", "C.I.L Number", "Total à Payer (Dirhams)", "Date Limite de Paiement", "Numéro de Client"]

if uploaded_files:
    for uploaded_file in uploaded_files:
        pdf_reader = PyPDF2.PdfReader(uploaded_file)
        text = ""
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()

        # Extract required information using Regex
        cil_pattern = r"(\d{7}-\d{3})"
        total_payer_pattern = r"(\d{3,4},\d{2})"
        date_limite_pattern = r"(\d{2}-\d{2}-\d{4})"

        format_matches = re.findall(cil_pattern, text)
        total_payer_match = re.search(total_payer_pattern, text)
        date_limite_match = re.search(date_limite_pattern, text)

        cil_number = format_matches[-1] if len(format_matches) > 1 else "Not found"
        numero_client = format_matches[0] if len(format_matches) > 1 and format_matches[0] != cil_number else "Not found"
        total_payer = total_payer_match.group(1) if total_payer_match else "Not found"
        date_limite = date_limite_match.group(1) if date_limite_match else "Not found"

        results.append([uploaded_file.name, cil_number, total_payer, date_limite, numero_client])

    df = pd.DataFrame(results, columns=headers)

    # Display data in a table format
    st.write(df)

    # Generate download link for Excel
    st.markdown(get_table_download_link(df), unsafe_allow_html=True)


Overwriting streamlit_app.py


# **Run Streamlit on Google Colab**

In [24]:
import os

# Download and unzip ngrok only if it doesn't exist yet
if not os.path.exists("ngrok-stable-linux-amd64.zip"):
    !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
    !unzip -o ngrok-stable-linux-amd64.zip

# Authenticate with ngrok using your auth token
!./ngrok authtoken 2Wtkkzkovj51i2fQ8MUCQrf7DyT_7cft4ztcihL3M9MvpsKdC


Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [25]:
# Start ngrok for the Streamlit port 8501
get_ipython().system_raw('./ngrok http 8501 &')
!sleep 2  # Wait a couple of seconds for the tunnel to be established

# Fetch and print the public URL
public_url = !curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; tunnels = json.load(sys.stdin)['tunnels']; print(tunnels[0]['public_url'] if tunnels else '')"
print(public_url[0])


https://82f7-34-42-64-2.ngrok-free.app


In [ ]:
!streamlit run streamlit_app.py





  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.42.64.2:8501

/content/streamlit_app.py:15: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
/content/streamlit_app.py:15: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
